In [3]:
import numpy as np

#from assets.keras.keras.datasets import shapes_3d
from assets.keras.keras.preprocessing.image import ImageDataGenerator
from assets.keras.keras.models import Sequential
from assets.keras.keras.layers.core import Dense, Dropout, Activation, Flatten
from assets.keras.keras.layers.convolutional import Convolution3D, MaxPooling3D, ZeroPadding3D
from assets.keras.keras.optimizers import SGD, RMSprop
from assets.keras.keras.utils import np_utils, generic_utils
from assets.keras.keras.regularizers import l2
import theano

#load dataset from gen_dataset
import cPickle as pickle

Using Theano backend.
/root/anaconda2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")
Using Theano backend.


In [4]:
data = pickle.load(open('assets/data/lr_ds.data','rb'))



In [5]:


X_train = data['X_train']
X_train = np.expand_dims(X_train,axis=1)
Y_train = data['y_train']
X_val = data['X_val']
X_val = np.expand_dims(X_val,axis=1)
Y_val = data['y_val']
X_test = data['X_test']
X_test = np.expand_dims(X_test,axis=1)
Y_test = data['y_test']
print 'Train data shape: ', X_train.shape
print 'Train labels shape: ', Y_train.shape
print 'Validation data shape: ', X_val.shape
print 'Validation labels shape: ', Y_val.shape
print 'Test data shape: ', X_test.shape
print 'Test labels shape: ', Y_test.shape

###
X_train = X_train[0:200,:,:,:,:]
Y_train = Y_train[0:200]
X_val = X_val[0:40,:,:,:,:]
Y_val = Y_val[0:40]
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train-1, 2)
Y_val = np_utils.to_categorical(Y_val-1,2)
Y_test = np_utils.to_categorical(Y_test-1, 2)



Train data shape:  (800, 1, 16, 32, 32)
Train labels shape:  (800,)
Validation data shape:  (100, 1, 16, 32, 32)
Validation labels shape:  (100,)
Test data shape:  (100, 1, 16, 32, 32)
Test labels shape:  (100,)


In [ ]:


###
###
__author__ = 'Dan Birman'



"""
    To classify/track 3D shapes, such as human hands (http://www.dbs.ifi.lmu.de/~yu_k/icml2010_3dcnn.pdf),
    we first need to find a distinct set of features. Specifically for 3D shapes, robust classification can be done using
    3D features.

    Features can be extracted by applying a 3D filters. We can auto learn these filters using 3D deep learning.

    This example trains a simple network for classifying 3D shapes (Spheres, and Cubes).

    GPU run command:
        THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python shapes_3d_cnn.py

    CPU run command:
        THEANO_FLAGS=mode=FAST_RUN,device=cpu,floatX=float32 python shapes_3d_cnn.py

    For 4000 training samples and 1000 test samples.
    90% accuracy reached after 40 epochs, 37 seconds/epoch on GTX Titan
"""

# Data Generation parameters
#test_split = 0.2
#dataset_size = 100
#patch_size = 16



In [6]:

#(X_train, Y_train),(X_test, Y_test) = shapes_3d.load_data(test_split=test_split,
#                                                          dataset_size=dataset_size,
#                                                          patch_size=patch_size)

print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print('X_val shape:', X_val.shape)
print('Y_val shape:', Y_val.shape)
print('X_test shape:', X_test.shape)
print('Y_test shape:', Y_test.shape)
print(X_train.shape[0], 'train samples')
print(X_val.shape[0], 'val samples')
print(X_test.shape[0], 'test samples')



('X_train shape:', (200, 1, 16, 32, 32))
('Y_train shape:', (200, 2))
('X_val shape:', (40, 1, 16, 32, 32))
('Y_val shape:', (40, 2))
('X_test shape:', (100, 1, 16, 32, 32))
('Y_test shape:', (100, 2))
(200, 'train samples')
(40, 'val samples')
(100, 'test samples')


In [46]:

# CNN Training parameters
batch_size = 40
nb_classes = 2
nb_epoch = 100


# number of convolutional filters to use at each layer
nb_filters = [8,8,32]

# level of pooling to perform at each layer (POOL x POOL)
nb_pool = [4,4,4]

# level of convolution to perform at each layer (CONV x CONV)
depth = [3,3,3]
nb_conv = [3,3,3]

# Regularization
reg = 0

model = Sequential()
model.add(ZeroPadding3D((1,1,1),input_shape=(1,16,32,32)))
model.add(Convolution3D(nb_filters[0],len_conv_dim1=nb_conv[0], len_conv_dim2=nb_conv[0], len_conv_dim3=nb_conv[0], border_mode='valid',
                         activation='relu', W_regularizer=l2(reg),dim_ordering='th'))
model.add(MaxPooling3D(pool_size=(1, nb_pool[0], nb_pool[0])))
#model.add(Dropout(0.5))
#model.add(ZeroPadding3D((1,1,1)))
#model.add(Convolution3D(nb_filters[1],len_conv_dim1=nb_conv[1], len_conv_dim2=nb_conv[1], len_conv_dim3=nb_conv[1], border_mode='valid',
#                        activation='relu', W_regularizer=l2(reg)))
#model.add(MaxPooling3D(pool_size=(nb_pool[1], nb_pool[1], nb_pool[1])))
#model.add(Dropout(0.5))
#model.add(ZeroPadding3D((1,1,1)))
#model.add(Convolution3D(nb_filters[2],nb_depth=nb_conv[2], nb_row=nb_conv[2], nb_col=nb_conv[2], border_mode='valid',
#                        activation='relu', W_regularizer=l2(reg)))
model.add(Flatten())
#model.add(Dropout(0.5))
model.add(Dense(16, init='normal', activation='relu', W_regularizer=l2(reg)))
model.add(Dense(nb_classes, init='normal', W_regularizer=l2(reg)))
model.add(Activation('softmax'))

sgd = RMSprop(lr=1e-2, decay=0, momentum=0.9, nesterov=True)
model.compile(loss='mse', optimizer=sgd)

hist = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=True, verbose=2,
          validation_data=(X_val, Y_val))
score = model.evaluate(X_test, Y_test, batch_size=batch_size, show_accuracy=True)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 200 samples, validate on 40 samples
Epoch 1/100
2s - loss: 0.4850 - acc: 0.5150 - val_loss: 0.5000 - val_acc: 0.5000
Epoch 2/100
2s - loss: 0.5000 - acc: 0.5000 - val_loss: 0.5000 - val_acc: 0.5000
Epoch 3/100
2s - loss: 0.5000 - acc: 0.5000 - val_loss: 0.5000 - val_acc: 0.5000
Epoch 4/100
2s - loss: 0.5000 - acc: 0.5000 - val_loss: 0.5000 - val_acc: 0.5000
Epoch 5/100
2s - loss: 0.5000 - acc: 0.5000 - val_loss: 0.5000 - val_acc: 0.5000
Epoch 6/100
2s - loss: 0.5000 - acc: 0.5000 - val_loss: 0.5000 - val_acc: 0.5000
Epoch 7/100
2s - loss: 0.5000 - acc: 0.5000 - val_loss: 0.5000 - val_acc: 0.5000
Epoch 8/100
2s - loss: 0.5000 - acc: 0.5000 - val_loss: 0.5000 - val_acc: 0.5000
Epoch 9/100
2s - loss: 0.5000 - acc: 0.5000 - val_loss: 0.5000 - val_acc: 0.5000
Epoch 10/100
2s - loss: 0.5000 - acc: 0.5000 - val_loss: 0.5000 - val_acc: 0.5000
Epoch 11/100
2s - loss: 0.5000 - acc: 0.5000 - val_loss: 0.5000 - val_acc: 0.5000
Epoch 12/100
2s - loss: 0.5000 - acc: 0.5000 - val_loss: 0.500

KeyboardInterrupt: 

In [45]:
for layer in model.layers:
    g = layer.get_config()
    h = layer.get_weights()
    print g
    for hi in h:
        print hi
        print hi.shape

{'name': 'ZeroPadding3D', 'custom_name': 'zeropadding3d', 'trainable': True, 'input_shape': (1, 16, 32, 32), 'padding': (1, 1, 1), 'cache_enabled': True}
{'len_conv_dim3': 3, 'W_constraint': None, 'b_constraint': None, 'name': 'Convolution3D', 'custom_name': 'convolution3d', 'len_conv_dim1': 3, 'activation': 'relu', 'trainable': True, 'dim_ordering': 'th', 'subsample': (1, 1, 1), 'cache_enabled': True, 'init': 'glorot_uniform', 'nb_filter': 8, 'len_conv_dim2': 3, 'b_regularizer': None, 'W_regularizer': {'l2': 0, 'name': 'WeightRegularizer', 'l1': 0.0}, 'activity_regularizer': None, 'border_mode': 'valid'}
[[[[[ -2.74358571e-01   3.72564256e-01   3.87477309e-01]
    [  3.94454718e-01   1.85320616e-01   1.63787410e-01]
    [  1.35764420e-01  -1.26158055e-02  -3.69140923e-01]]

   [[ -3.71914744e-01   1.13810241e-01   2.24584281e-01]
    [  1.73066974e-01  -2.85709143e-01  -9.43891332e-03]
    [  5.10011315e-02  -1.17257498e-02   8.13381821e-02]]

   [[  4.04367119e-01  -1.45675093e-01   

In [52]:
print X_train.shape
print X_train[0,0,:,0:5,0]

(200, 1, 16, 32, 32)
[[  0   0   0   0   0]
 [  0   0 255   0   0]
 [  0   0   0 255   0]
 [  0   0   0   0   0]
 [  0   0   0   0   0]
 [  0   0   0 255 255]
 [255   0 255   0   0]
 [  0   0   0   0   0]
 [255   0   0   0 255]
 [  0   0   0   0   0]
 [  0   0   0   0 255]
 [  0   0 255 255 255]
 [  0   0   0   0   0]
 [255   0   0   0   0]
 [  0 255 255   0 255]
 [  0   0   0   0   0]]


In [ ]:
print dir(model.layers[3])